# Predictions

In [2]:
import pandas as pd
import numpy as np
# import pickle

df = pd.read_csv('../data/train.csv')

/Users/balazsfeher/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['day'] = pd.DatetimeIndex(df['Date']).day
df.drop(['Date', 'Customers'], axis=1, inplace=True)

In [4]:
target_col = ['Sales']
feature_col = ['year', 'month', 'day']
X_all = df[feature_col]
y_all = df[target_col]

In [5]:
X_all = pd.get_dummies(X_all.astype(str))

In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib

scaler = MinMaxScaler()
y_all = scaler.fit_transform(y_all.apply(np.log1p))
# Save scaler
joblib.dump(scaler, '../model_deployment/scaler.pkl')
X_all = X_all.as_matrix()

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=10)

In [7]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.layers.advanced_activations import PReLU

model = Sequential()
model.add(Dense(50, input_dim=X_train.shape[1], init='he_normal'))
model.add(PReLU())

model.add(Dense(50, init='he_normal'))
model.add(PReLU())

model.add(Dense(1, init='he_normal'))
model.add(Activation('sigmoid'))

adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss='mse', optimizer=adam)

Using TensorFlow backend.
/Users/balazsfeher/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, input_dim=46, kernel_initializer="he_normal")`
  import sys
/Users/balazsfeher/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, kernel_initializer="he_normal")`
  # Remove the CWD from sys.path while we load stuff.
/Users/balazsfeher/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="he_normal")`
  del sys.path[0]


In [8]:
model.fit(X_train, y_train, batch_size=128, validation_split=0.1, epochs=1, verbose=1)

Train on 732390 samples, validate on 81377 samples
Epoch 1/1
732390/732390 [==============================] - 8s - loss: 0.0148 - val_loss: 0.0067


In [9]:
# Serialize model to JSON
model_json = model.to_json()
with open("../model_deployment/model.json", "w") as json_file:
    json_file.write(model_json)
# Serialize weights to HDF5
model.save_weights("../model_deployment/model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# sample_input = [[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
#                  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
#                  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
#                  0.,  0.,  0.,  0.,  0.,  0.,  0.]]